In [0]:
# Load the dataset from DBFS (Databricks Filesystem)
file_path = "/databricks-datasets/samples/population-vs-price/data_geo.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)
df.printSchema()
df.show(5)

root
 |-- 2014 rank: integer (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- State Code: string (nullable = true)
 |-- 2014 Population estimate: integer (nullable = true)
 |-- 2015 median sales price: double (nullable = true)

+---------+-------------+-------+----------+------------------------+-----------------------+
|2014 rank|         City|  State|State Code|2014 Population estimate|2015 median sales price|
+---------+-------------+-------+----------+------------------------+-----------------------+
|      101|   Birmingham|Alabama|        AL|                  212247|                  162.9|
|      125|   Huntsville|Alabama|        AL|                  188226|                  157.7|
|      122|       Mobile|Alabama|        AL|                  194675|                  122.5|
|      114|   Montgomery|Alabama|        AL|                  200481|                  129.0|
|       64|Anchorage[19]| Alaska|        AK|                  30101

In [0]:
# Select relevant columns for analysis
df_selected = df.select(
    "City", "State", "State Code", 
    "2014 Population estimate", "2015 median sales price"
)
df_selected.show(5)

+-------------+-------+----------+------------------------+-----------------------+
|         City|  State|State Code|2014 Population estimate|2015 median sales price|
+-------------+-------+----------+------------------------+-----------------------+
|   Birmingham|Alabama|        AL|                  212247|                  162.9|
|   Huntsville|Alabama|        AL|                  188226|                  157.7|
|       Mobile|Alabama|        AL|                  194675|                  122.5|
|   Montgomery|Alabama|        AL|                  200481|                  129.0|
|Anchorage[19]| Alaska|        AK|                  301010|                   null|
+-------------+-------+----------+------------------------+-----------------------+
only showing top 5 rows



In [0]:
# TOP 10 MOST POPULOUS CITIES
from pyspark.sql.functions import desc
populated_cities=df.select("City").orderBy(desc("2014 population estimate")).limit(10)
display(populated_cities)

City
New York[6]
Los Angeles
Chicago
Houston[7]
Philadelphia[8]
Phoenix
San Antonio
San Diego
Dallas
San Jose


Databricks visualization. Run in Databricks to view.

In [0]:
#AVERAGE MEDIAN SALES PRICE BY STATE
from pyspark.sql.functions import avg

# Group by State and calculate average of "2015 median sales price"
avg_price_by_state = df_selected.groupBy("State").agg(
    avg("2015 median sales price").alias("Average_Median_Sales_Price")
)

# Show the result
avg_price_by_state.orderBy("Average_Median_Sales_Price", ascending=False).show(10)


+--------------------+--------------------------+
|               State|Average_Median_Sales_Price|
+--------------------+--------------------------+
|             Hawai'i|                     699.3|
|          California|         547.9714285714286|
|District of Columbia|                     367.8|
|          New Jersey|                     350.8|
|            Colorado|        333.46666666666664|
|          Washington|        266.29999999999995|
|       Massachusetts|                     261.8|
|         Connecticut|        250.13333333333333|
|                Utah|                     243.3|
|       New Hampshire|                     237.4|
+--------------------+--------------------------+
only showing top 10 rows



In [0]:
#Filter cities with population > 1 Millon and display in descending order
population = df_selected.filter(df_selected["2014 Population estimate"] > 1_000_000)
display(population.orderBy("2014 Population estimate", ascending=False))


City,State,State Code,2014 Population estimate,2015 median sales price
New York[6],New York,NY,8491079,388.6
Los Angeles,California,CA,3928864,434.7
Chicago,Illinois,IL,2722389,192.5
Houston[7],Texas,TX,2239558,200.3
Philadelphia[8],Pennsylvania,PA,1560297,204.9
Phoenix,Arizona,AZ,1537058,206.1
San Antonio,Texas,TX,1436697,184.7
San Diego,California,CA,1381069,510.3
Dallas,Texas,TX,1281047,192.5
San Jose,California,CA,1015785,900.0


In [0]:
#Identify cities with missing or zero price values
#Identify cities with missing or zero price values
df_selected.filter((df_selected["2015 median sales price"].isNull()) | (df_selected["2015 median sales price"] == 0)).show()

  File <command-314138678797492>:3
    df_selected.filter((df_selected["2015 median sales price"].isNull()) | (df_selected["2015 median sales price"] == 0)).show()
                                                                                                                  ^
SyntaxError: invalid non-printable character U+00A0
